In [ ]:
!pwd

/content/mydrive/MyDrive/SSN FDP/Day6/Data


In [ ]:
from google.colab import drive
drive.mount('/content/mydrive')

Drive already mounted at /content/mydrive; to attempt to forcibly remount, call drive.mount("/content/mydrive", force_remount=True).


In [ ]:
from google.colab import drive
import os
#drive.mount('/content/mydrive')
os.chdir(os.path.join('/','content','mydrive', 'MyDrive','SSN FDP', 'Day6', 'Data'))
!ls
#/content/mydrive/MyDrive/SSN FDP/Day6/Data

cnn_histopath_model_save_demo.h5  histopathology.ipynb	mini_trainset
cnn_histopath_weights_demo.hdf5   mini_testset_histo


# New section

In [ ]:
# Part 1 - Building the CNN

# Importing the Keras libraries and packages
!pip install visualkeras
import visualkeras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, BatchNormalization
#from keras.layers.normalization import BatchNormalization
from keras.layers import SeparableConv2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dropout, Dense
from keras import backend as K


import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adagrad, Adadelta, Nadam
#from keras.optimizers import Adagrad Adadelta, Nadam
from keras.utils import np_utils
from sklearn.metrics import classification_report, confusion_matrix
from imutils import paths
import numpy as np
import os

num_cores=12
size_batch=4#16
use_multi_proc =False

In [ ]:
def get_model(width, height, depth, classes):
  model = Sequential()    # initialize model
  inputShape = (height, width, depth)
  channelDim = -1
                                      
  if K.image_data_format() == 'channels_first':
      inputShape = (depth, height, width)
      channelDim = 1 

  model.add(SeparableConv2D(32, (3,3), padding='same', input_shape=inputShape))
  model.add(Activation('relu'))
  model.add(BatchNormalization(axis=channelDim))
  model.add(MaxPooling2D(pool_size=(2,2)))


  model.add(SeparableConv2D(64, (3,3), padding='same'))
  model.add(Activation('relu'))
  model.add(BatchNormalization(axis=channelDim))
  model.add(MaxPooling2D(pool_size=(2,2)))

  model.add(SeparableConv2D(64, (3,3), padding='same'))
  model.add(Activation('relu'))
  model.add(BatchNormalization(axis=channelDim))
  model.add(MaxPooling2D(pool_size=(2,2)))
  #model.add(Dropout(0.25))

  # CONV2D(128) -> RELU -> BATCH NORM -> CONV2D(128) -> RELU -> BATCH NORM -> CONV2D(128) -> RELU -> BATCH NORM
  # -> POOL2D -> DROPOUT
  model.add(SeparableConv2D(128,(3,3), padding='same'))
  model.add(Activation('relu'))
  model.add(BatchNormalization(axis=channelDim))
  model.add(MaxPooling2D(pool_size=(2,2)))

 

  # FC(256) -> RELU -> BATCH NORM -> DROPOUT
  model.add(Flatten())
  model.add(Dense(256))
  model.add(Activation('relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.5))

    # Softmax clqssifier
  model.add(Dense(classes))
  model.add(Activation('softmax'))
  visualkeras.layered_view(model).show()
  return model

In [ ]:
number_of_train = 1670
number_of_test = 300
numEpochs = 20
batchSize = 32
lrRate = 1e-2
lrRateDecay = lrRate/numEpochs

train_data_dir = 'mini_trainset/'
test_data_dir = 'mini_testset_histo/'

            
train_list = list(paths.list_images(train_data_dir))
test_list = list(paths.list_images(test_data_dir))
  #Initializing training data augmentation object
trainAug = ImageDataGenerator(rotation_range=20,
                            rescale=1/255.0,
                            zoom_range=0.05,
                            height_shift_range=0.1,
                            width_shift_range=0.1,
                            horizontal_flip=True,
                            vertical_flip=True,
                            shear_range=0.5,
                            fill_mode='nearest')
testAug = ImageDataGenerator(rescale=1/255.0)
#Initializing training and testing generator
trainGen = trainAug.flow_from_directory(train_data_dir,
                                        class_mode='categorical',
                                        target_size=(128,128), 
                                        shuffle=True,                                                                                           
                                        batch_size=batchSize,
                                        color_mode='rgb')

#model building
model = get_model(width=128, height=128, depth=3, classes=3)
model.summary()
opt = Nadam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)

#model compilation
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

#to store the model after every epoch
#callbacks = [ModelCheckpoint(filepath='histopath_weights.{epoch:02d}-{val_loss:.2f}.h5')]   


#fit the model
history = model.fit_generator(trainGen,
                            steps_per_epoch=number_of_train // batchSize,
                            epochs=numEpochs)

model.save('cnn_histopath_model_save_demo.h5')
model.save_weights('cnn_histopath_weights_demo.hdf5')
print('Training complete')

Found 733 images belonging to 3 classes.
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 separable_conv2d_8 (Separab  (None, 128, 128, 32)     155       
 leConv2D)                                                       
                                                                 
 activation_12 (Activation)  (None, 128, 128, 32)      0         
                                                                 
 batch_normalization_10 (Bat  (None, 128, 128, 32)     128       
 chNormalization)                                                
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 separable_conv2d_9 (Separab  (None, 64, 64, 64)       2400      
 leConv2D)   

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/nadam.py:73: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Nadam, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/20
52/52 [==============================] - 30s 505ms/step - loss: 0.6112 - accuracy: 0.8035
Training complete


# New Section

In [ ]:
testGen = testAug.flow_from_directory(test_data_dir,
                                    class_mode='categorical',
                                    target_size=(128,128),
                                    shuffle=False,
                                    batch_size=batchSize,
                                    color_mode='rgb')

print(len(testGen.classes))
print(testGen.class_indices.keys())

Found 300 images belonging to 3 classes.
300
dict_keys(['lung_aca', 'lung_n', 'lung_scc'])


In [ ]:
number_of_train = 1670
number_of_test = 300
numEpochs = 20
batchSize = 32
lrRate = 1e-2
lrRateDecay = lrRate/numEpochs

train_data_dir = 'mini_trainset/'
test_data_dir = 'mini_testset_histo/'

            
train_list = list(paths.list_images(train_data_dir))
test_list = list(paths.list_images(test_data_dir))
  #Initializing training data augmentation object
trainAug = ImageDataGenerator(rotation_range=20,
                            rescale=1/255.0,
                            zoom_range=0.05,
                            height_shift_range=0.1,
                            width_shift_range=0.1,
                            horizontal_flip=True,
                            vertical_flip=True,
                            shear_range=0.5,
                            fill_mode='nearest')
testAug = ImageDataGenerator(rescale=1/255.0)
#Initializing training and testing generator
trainGen = trainAug.flow_from_directory(train_data_dir,
                                        class_mode='categorical',
                                        target_size=(128,128), 
                                        shuffle=True,                                                                                           
                                        batch_size=batchSize,
                                        color_mode='rgb')

#model building
model = get_model(width=128, height=128, depth=3, classes=3)
model.summary()
opt = Nadam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)

#model compilation
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

#to store the model after every epoch
#callbacks = [ModelCheckpoint(filepath='histopath_weights.{epoch:02d}-{val_loss:.2f}.h5')]   


#fit the model
history = model.fit_generator(trainGen,
                            steps_per_epoch=number_of_train // batchSize,
                            epochs=numEpochs)

model.save('cnn_histopath_model_save_demo.h5')
model.save_weights('cnn_histopath_weights_demo.hdf5')
print('Training complete')

Found 733 images belonging to 3 classes.
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 separable_conv2d_12 (Separa  (None, 128, 128, 32)     155       
 bleConv2D)                                                      
                                                                 
 activation_18 (Activation)  (None, 128, 128, 32)      0         
                                                                 
 batch_normalization_15 (Bat  (None, 128, 128, 32)     128       
 chNormalization)                                                
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 64, 64, 32)       0         
 g2D)                                                            
                                                                 
 separable_conv2d_13 (Separa  (None, 64, 64, 64)       2400      
 bleConv2D)  

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/nadam.py:73: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Nadam, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/20
52/52 [==============================] - 30s 505ms/step - loss: 0.6227 - accuracy: 0.7776
Training complete


In [ ]:
# make prediction on test data
predIdx = model.predict(testGen)
predIdx = np.argmax(predIdx, axis=1)

print(classification_report(testGen.classes,
                            predIdx,
                            target_names=testGen.class_indices.keys()))
confusionMatrix = confusion_matrix(testGen.classes, predIdx)
total = sum(sum(confusionMatrix))

              precision    recall  f1-score   support

    lung_aca       0.00      0.00      0.00       100
      lung_n       0.00      0.00      0.00       100
    lung_scc       0.33      1.00      0.50       100

    accuracy                           0.33       300
   macro avg       0.11      0.33      0.17       300
weighted avg       0.11      0.33      0.17       300



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
    # compute accuracy, sensitivty, specificity
    # sensitivity measures the proportion of true positives also predicted as positives
    # Similarly, specificity measures the proportion of true negatives
accuracy = (confusionMatrix[0,0] + confusionMatrix[1,1]) / total
sensitivity = confusionMatrix[0,0] / (confusionMatrix[0,0] + confusionMatrix[0,1])
specificity = confusionMatrix[1,1] / (confusionMatrix[1,0] + confusionMatrix[1,1])
print(confusionMatrix)
print('accuracy: {:4f}'.format(accuracy))
print('sensitivity: {:4f}'.format(sensitivity))
print('specificity: {:4f}'.format(specificity))

[[  0   0 100]
 [  0   0 100]
 [  0   0 100]]
accuracy: 0.000000
sensitivity:  nan
specificity:  nan


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in long_scalars
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in long_scalars
  
